In [ ]:
# Install dependencies
!pip install transformers datasets tokenizers sentencepiece torch sentence-transformers tqdm nltk pandas numpy
import nltk
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Import required datasets
import datasets

# # Load the SQUAD dataset
# squad_dataset = datasets.load_dataset('squad')

# # Split the dataset into train and test sets
# train_dataset = squad_dataset['train']
# validation_dataset = squad_dataset['validation']

# Load CoQA dataset
coqa_dataset = datasets.load_dataset('coqa')
coqa_train_dataset = coqa_dataset['train']
coqa_validation_dataset = coqa_dataset['validation']

# Load MLQA
mlqa_dataset = datasets.load_dataset('mlqa', 'mlqa.en.en')
mlqa_test_dataset = mlqa_dataset['test']
mlqa_validation_dataset = mlqa_dataset['validation']

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def convert_datasets_to_list(dataset):
  return dataset.select(range(len(dataset))).to_pandas().values.tolist()

In [ ]:
# Prepare and preprocess coqa dataset
import nltk
import pandas as pd

question_answer_pairs = []

coqa_train_dataset_list = convert_datasets_to_list(coqa_train_dataset)
coqa_validation_dataset_list = convert_datasets_to_list(coqa_validation_dataset)
coqa_dataset_list = coqa_train_dataset_list + coqa_validation_dataset_list

mlqa_test_dataset_list = convert_datasets_to_list(mlqa_test_dataset)
mlqa_validation_dataset_list = convert_datasets_to_list(mlqa_validation_dataset)
mlqa_dataset_list = mlqa_test_dataset_list + mlqa_validation_dataset_list

question_answer_context_df = pd.DataFrame(columns=["question", "answer", "passage", "source"])

question_start = ["what", "how", 'when', 'why', 'where', 'is']

for data in coqa_dataset_list:
  questions = data[2]
  for question_index, question in enumerate(questions):
    answer = data[3]['input_text'][question_index]
    tokenized_answer = nltk.tokenize.word_tokenize(answer)
    tokenized_question = nltk.tokenize.word_tokenize(question)
    if answer != "unknown" and len(tokenized_answer) <= 3 and len(tokenized_question) > 3 and question.lower().split()[0] in question_start:
      question_answer_context_df = question_answer_context_df.append({
          "question": question,
          "answer": answer,
          "passage": data[1].replace("\n", ""),
          "source": "coqa"
      }, ignore_index=True)

# Prepare and preprocess mlqa dataset
for data in mlqa_dataset_list:
  question_answer_context_df = question_answer_context_df.append({
      "question": data[1],
      "answer": data[2]["text"],
      "passage": data[0].replace("\n", ""),
      "source": "mlqa"
  }, ignore_index=True)

# Export dataframe as csv
question_answer_context_df.to_json('question_answer_context.json', orient="records")

In [ ]:
# Randomly sample 1000 data from each source
sampled_df = question_answer_context_df.groupby("source").sample(1000)
sampled_df.to_json('sampled_question_answer_context.json', orient="records")